In [67]:
from pathlib import Path
import os
import csv

In [83]:
failedRuns = []
pathlist = Path("../logs").glob('**/*.out')
for path in pathlist:
     # because path is object not string
    path_in_str = str(path)
    with open(path_in_str, 'r') as f:
        for line in f.readlines():
            if 'core dumped' in line:
                toParse = line
                splitLine = toParse.split()
                runline = splitLine[len(splitLine)-2]
                runline=runline.strip("/fs/scratch/PAS0654/jorge/ARA_data/")
                failedRun = runline.partition("event00")[2].strip(".")
#                 print(failedRun)
                failedRuns.append(failedRun)
print("%i failed runs"%len(failedRuns))
failedRuns.sort()
print(failedRuns)

28 failed runs
['7536', '7550', '7552', '7553', '7587', '7589', '7618', '7698', '7712', '7714', '7719', '7803', '7834', '7835', '7853', '7856', '7857', '7875', '7877', '7879', '7880', '7883', '7884', '7885', '7886', '7888', '7931', '7932']


In [100]:
project = "PCON0003" #what project to use
station = 2
year = 2016
print("Year: %i"%year)
isSim = 0 #data (0) or simulation (1)
anaFolder = os.path.expanduser('/users/PAS0654/osu8354/ARA_cvmfs/source/AraRoot/analysis/ARA_analysis/ARA_SourceSearch/') #Where are the files with the run lists
step1_makePairs = anaFolder + "OSC_scripts/step1-make_ped_pairs/"
outputDir = "/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/CWID/A%i/%i"%(station, year)
summaryDir = "/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A%i/%i"%(station, year)
file = step1_makePairs+"A%i_%i_File_Ped_Pairs.txt"%(station,year)

with open(file, mode='r', newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile)
    count_core = 0
    for job in reader:
        split = os.path.split(job[0])
        run = os.path.splitext(split[1])[0].lstrip("event00")
        if(run not in failedRuns):
            continue
#         print(run)
        submit_command = ("sbatch "
                        "--job-name=ReDo_CWID_{0}_{7} --output=./logs/{7}_CWID_{0}.out --account={6} "
                        "--export=ISSIM={1},STATION={2},OUTDIR={5},YEAR={7},SUMMARYDIR={3},DATA={8},PEDESTAL={4} OSCReSubmit.sh").format(run,isSim,station,summaryDir,job[1],outputDir, project, year, job[0])#Add additional core for memory
        print(submit_command)
        break

Year: 2016
sbatch --job-name=ReDo_CWID_7536_2016 --output=./logs/2016_CWID_7536.out --account=PCON0003 --export=ISSIM=0,STATION=2,OUTDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/CWID/A2/2016,YEAR=2016,SUMMARYDIR=/fs/project/PAS0654/ARA_DATA/A23/10pctCalibBugFix/RunSummary/A2/2016,DATA=/fs/scratch/PAS0654/jorge/ARA_data/10pct/A2/2016/0704/run007536/event007536.root,PEDESTAL=/fs/project/PAS0654/ARA_DATA/A23/peds/A2/2016/pedestalValues.run007531.dat OSCReSubmit.sh


In [97]:
job[0]

'/fs/scratch/PAS0654/jorge/ARA_data/10pct/A2/2016/0704/run007536/event007536.root'